# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 27 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
!nvidia-smi

Fri Feb 17 02:29:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torchsummaryX wandb --quiet

In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
import math
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!pip install --upgrade --force-reinstall --no-deps kaggle 
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"shekennotstirred","key":"c0878af19b47a450335ad68980815833"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# commands to download data from kaggle

! cd /content/drive/MyDrive/11-785-IDL/HW/HW1/P2/data
! ls -a 
!kaggle competitions download -c 11-785-s23-hw1p2
!mkdir '/content/data/'
!unzip -qo '11-785-s23-hw1p2.zip' -d '/content/data/'


/bin/bash: line 0: cd: /content/drive/MyDrive/11-785-IDL/HW/HW1/P2/data: No such file or directory
.  ..  11-785-s23-hw1p2.zip  .config  sample_data
11-785-s23-hw1p2.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import shutil
import os

src_dir = '/content/data/11-785-s23-hw1p2/train-clean-100/mfcc/'
dest_dir = '/content/data/11-785-s23-hw1p2/train-clean-460/mfcc/'

shutil.copytree

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
"""
MEMORY EFFICIENT DATASET FOR QUICKER ACCESS
"""
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, partition, context, phonemes): # Feel free to add more arguments

        self.context = context
        self.phonemes = phonemes
        self.root = root
        self.partition = partition
        
        # MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = self.root + self.partition + '/' + 'mfcc/'
        # Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = self.root + self.partition + '/' + 'transcript/'

        # List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir)) 
        # List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(os.listdir(self.transcript_dir))
        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        T = 0
        for i in range(len(mfcc_names)):
          mfcc = np.load(self.mfcc_dir + mfcc_names[i])
          T += mfcc.shape[0]
        
        self.mfccs = np.zeros(((T + (2*self.context)), mfcc.shape[1]), dtype=np.float32)
        self.transcripts = np.zeros((T,), dtype=np.uint8)

        cx, cy = self.context, 0

        # Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):

            mfcc = np.load(self.mfcc_dir + mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
        
        #   Load the corresponding transcript
            # Remove [SOS] and [EOS] from the transcript 
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
            # Map the String phoneme value to integer values for our model
            transcript  = np.load(self.transcript_dir + transcript_names[i])[1:-1]
            transcript = np.array([self.phonemes.index(phoneme) for phoneme in transcript])

            T_i = mfcc.shape[0]
            self.mfccs[cx:cx+T_i] = mfcc
            self.transcripts[cy:cy+T_i] = transcript

            cx += T_i
            cy += T_i

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = T

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        # Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        # Flatten to get 1d data
        frames = self.mfccs[ind:ind+2*self.context+1, :].flatten()
        # frames = self.mfccs[ind-self.context:ind+self.context+1, :].flatten()
        # print(frames.shape)
        frames = torch.FloatTensor(frames)
        phonemes = torch.tensor(self.transcripts[ind])       

        return frames, phonemes

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):    
    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, root, context, phonemes, partition): # Feel free to add more arguments

      self.context = context
      self.phonemes = phonemes
      self.root = root
      
      # MFCC directory - use partition to acces train/dev directories from kaggle data using root
      self.mfcc_dir       = self.root + partition + '/' + 'mfcc/'

      # List files in sefl.mfcc_dir using os.listdir in sorted order
      mfcc_names          = sorted(os.listdir(self.mfcc_dir)) 

      T = 0
      for i in range(len(mfcc_names)):
        mfcc = np.load(self.mfcc_dir + mfcc_names[i])
        T += mfcc.shape[0]
      
      self.mfccs = np.zeros(((T + (2*self.context)), mfcc.shape[1]), dtype=np.float32)

      cx = self.context

      # Iterate through mfccs and transcripts
      for i in range(len(mfcc_names)):
          mfcc = np.load(self.mfcc_dir + mfcc_names[i])
      #   Do Cepstral Normalization of mfcc (explained in writeup)
          mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)

      #   add mfcc to array
          T_i = mfcc.shape[0]
          self.mfccs[cx:cx+T_i] = mfcc
          cx += T_i

      # Length of the dataset is now the length of concatenated mfccs
      self.length = T

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        # Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        # Flatten to get 1d data
        frames = self.mfccs[ind:ind+2*self.context+1, :].flatten()
        frames = torch.FloatTensor(frames)
        return frames

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [ ]:
"""
Thomas's Final Arch for 360 = %
"""

config = {
    'epochs'        : 30,
    'batch_size'    : 1024,
    'context'       : 30,
    'init_lr'       : 1e-3,
    'architecture'  : 'high-cutoff',
    'train_dataset' : 'train-clean-360',
    'val_dataset'   : 'dev-clean',
    'test_dataset'  : 'test-clean',
    'dropout_rate'  : 0.2,
    'weight_decay'  : 1e-2,
    'patience'      : 2
}

# Create Datasets

In [ ]:
#TODO: Create a dataset object using the AudioDataset class for the training data 
root_dir = '/content/data/11-785-s23-hw1p2/'
train_data = AudioDataset(root=root_dir, partition=config['train_dataset'], phonemes=PHONEMES, context=config['context'])

# # TODO: Create a dataset object using the AudioDataset class for the validation data 
val_data = AudioDataset(root=root_dir, partition=config['val_dataset'], phonemes=PHONEMES, context=config['context'])

# # TODO: Create a dataset object using the AudioTestDataset class for the test data 
test_data = AudioTestDataset(root=root_dir, partition=config['test_dataset'], phonemes=PHONEMES, context=config['context'])

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 2, 
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  1024
Context        :  30
Input size     :  1647
Output symbols :  42
Train dataset samples = 130453995, batches = 127397
Validation dataset samples = 1928204, batches = 1884
Test dataset samples = 1934138, batches = 1889


In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(val_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([1024, 1647]) torch.Size([1024])


In [ ]:
# Testing code to check if your data loaders are working
val_phonemes = []
for i, val_data in enumerate(val_loader):
    val_frames, val_phoneme = val_data
    val_phonemes.append(val_phoneme.cpu().numpy())

# print(val_phonemes[0])
val_phonemes = np.concatenate(val_phonemes)
print(np.unique(val_phonemes, return_counts=True))
sil_index = np.argmax(np.unique(val_phonemes, return_counts=True)[1])
sil_count = np.max(np.unique(val_phonemes, return_counts=True)[1])
low_index = np.argmin(np.unique(val_phonemes, return_counts=True)[1])
low_count = np.min(np.unique(val_phonemes, return_counts=True)[1])
print(PHONEMES[sil_index])
print(sil_count)
print(PHONEMES[low_index])
print(low_count)


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
"""
Paul's Architecture 2 on full tc-360 = %
"""
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size, dropout_rate):

        super(Network, self).__init__()
        
        self.model = torch.nn.Sequential(
            
            #Input to Hidden layer 1 - removed dropout from first layer
            torch.nn.Linear(input_size, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(), # Layer 1

            #Hidden layer 2
            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(), # Layer 2
            torch.nn.Dropout(p=dropout_rate),


            #Hidden layer 3
            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(), # Layer 3
            torch.nn.Dropout(p=dropout_rate),

            #Hidden layer 4
            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(), # Layer 4
            torch.nn.Dropout(p=dropout_rate),

            #Hidden layer 5
            torch.nn.Linear(2048, 1288),
            torch.nn.BatchNorm1d(1288),
            torch.nn.GELU(), # Layer 5
            torch.nn.Dropout(p=dropout_rate),

            #Hidden layer 6
            
            torch.nn.Linear(1288, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(), # Layer 6
            torch.nn.Dropout(p=dropout_rate),

            #Linear Layer classifcation to 42 
            torch.nn.Linear(1024, output_size))
        
        self.apply(self._init_weights)

    def _init_weights(self, m):
      if isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight)
        torch.nn.init.zeros_(m.bias)
        
    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [ ]:
INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes),dropout_rate=config['dropout_rate']).to(device)
summary(model, frames.to(device))
# summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0         [1647, 2048]  [1024, 2048]  3.375104M  3.373056M
1_model.BatchNorm1d_1          [2048]  [1024, 2048]     4.096k     2.048k
2_model.GELU_2                      -  [1024, 2048]          -          -
3_model.Linear_3         [2048, 2048]  [1024, 2048]  4.196352M  4.194304M
4_model.BatchNorm1d_4          [2048]  [1024, 2048]     4.096k     2.048k
5_model.GELU_5                      -  [1024, 2048]          -          -
6_model.Dropout_6                   -  [1024, 2048]          -          -
7_model.Linear_7         [2048, 2048]  [1024, 2048]  4.196352M  4.194304M
8_model.BatchNorm1d_8          [2048]  [1024, 2048]     4.096k     2.048k
9_model.GELU_9                      -  [1024, 2048]          -          -
10_model.Dropout_10                 -  [1024, 2048]          -          -
11_model.Linear_11       [2048, 2048] 

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[1647, 2048]","[1024, 2048]",3375104.0,3373056.0
1_model.BatchNorm1d_1,[2048],"[1024, 2048]",4096.0,2048.0
2_model.GELU_2,-,"[1024, 2048]",NaN,NaN
3_model.Linear_3,"[2048, 2048]","[1024, 2048]",4196352.0,4194304.0
4_model.BatchNorm1d_4,[2048],"[1024, 2048]",4096.0,2048.0
5_model.GELU_5,-,"[1024, 2048]",NaN,NaN
6_model.Dropout_6,-,"[1024, 2048]",NaN,NaN
7_model.Linear_7,"[2048, 2048]","[1024, 2048]",4196352.0,4194304.0
8_model.BatchNorm1d_8,[2048],"[1024, 2048]",4096.0,2048.0


In [ ]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function. 
# We use CE because the task is multi-class classification 
# optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr']) #Defining Optimizer
optimizer = torch.optim.AdamW(params=model.parameters(), lr= config['init_lr'])

# optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr'], weight_decay=config['weight_decay']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate, 
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=config['patience'])
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high? 
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

1644

In [ ]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    
    for i, (frames, phonemes) in enumerate(dataloader):
        
        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward() 
        
        ### Gradient Descent
        optimizer.step()       

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))), 
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()
  
    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode(): 
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]
        
        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))), 
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()
    
        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="a948a7a6118c9cb14a779f2fbfbf9f4ea1b82754") #API Key is in your wandb account, under settings (wandb.ai/settings)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = "360-kaiming-decay0.01-0.2dropoutafterfirst-2plateau-1024-gelu-paul-arch2", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    id     = "hw1p2-mark7-train360", ### Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account 
    config  = config ### Wandb Config for your run
)

wandb: Currently logged in as: abhisheksingh-7 (deeper_learners). Use `wandb login --relogin` to force relogin


In [ ]:
### Save your model architecture as a string with str(model) 
model_arch  = str(model)

### Save it in a txt file 
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20230217_031608-hw1p2-mark7-train360/files/model_arch.txt']

In [ ]:
!mkdir '/content/checkpoint'
!ls '/content/'

mkdir: cannot create directory ‘/content/checkpoint’: File exists
11-785-s23-hw1p2.zip  checkpoint  data	model_arch.txt	sample_data  wandb


# Experiment

In [ ]:
checkpoint = torch.load('/content/checkpoint/gcp_model_checkpoint.pth23')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
torch.cuda.empty_cache()

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")
best_val_acc = 0

for epoch in range(18):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    #scheduler.step(val_loss)
    scheduler.step(val_loss)

    ### Log metrics at each epoch in your run 
    # Optionally, you can log at each batch inside train/eval functions 
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss, 
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    torch.save({'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_acc': val_acc},
          '/content/checkpoint/gcp_model_checkpoint.pth'+str(epoch+12))


Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
OSError: [Errno 9] Bad file descriptor



Epoch 1/30


Train:   0%|          | 0/127397 [00:00<?, ?it/s]

Val:   0%|          | 0/1884 [00:00<?, ?it/s]

	Train Acc 85.8707%	Train Loss 0.4109	 Learning Rate 0.0010000
	Val Acc 86.4861%	Val Loss 0.3908

Epoch 2/30


...

Train:   0%|          | 0/127397 [00:00<?, ?it/s]

Val:   0%|          | 0/1884 [00:00<?, ?it/s]

	Train Acc 89.7902%	Train Loss 0.2809	 Learning Rate 0.0000100
	Val Acc 88.7875%	Val Loss 0.3282

Epoch 14/30


Train:   0%|          | 0/127397 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)             
            
            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim= 1).cpu().numpy()

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
            # TODO
            test_predictions.append(predicted_phonemes)

            ### Release memory
            del mfccs, logits
            torch.cuda.empty_cache()

    return np.concatenate(test_predictions).tolist()

In [ ]:
state_dict = torch.load('/content/checkpoint/gcp_model_checkpoint.pth9')
model.load_state_dict(state_dict['model_state_dict'])

<All keys matched successfully>

In [ ]:
predictions = test(model, test_loader)

  0%|          | 0/1889 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f7fae247e50>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f7fae247e50>
Traceback (most recent call last):
      File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()self._shutdown_workers()

  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
        if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
if w.is_alive():assert self._parent_pid == os.getpid(), 'can only test a child process'    
AssertionError
: can only test a child process
  File "/usr/lib/pytho

In [ ]:
### Finish your wandb run
run.finish()

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, PHONEMES[predictions[i]]))
print(len(predictions))

1934138


In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11-785-s23-hw1p2 -f ./submission.csv -m "final"

### However, its always safer to download the csv file and then upload to kaggle

100% 19.3M/19.3M [00:00<00:00, 50.6MB/s]
Successfully submitted to Frame-Level Speech Recognition